# Cat - Dog Classifier

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend

backend.clear_session()

# Input Shape 
hx = 150 # width 
hy = 150 # height

if backend.image_data_format() == 'channels_first':
    input_shape = (3, hx, hy)
else:
    input_shape = (hx, hy, 3)


# Model Definition
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# save the model
model_json = model.to_json()
with open("cat-dog-vanilla-cnn.json", "w") as json_file:
    json_file.write(model_json)

In [29]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

batch_size = 16

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('data/train',
                                                   target_size=(hx, hy), 
                                                   batch_size=batch_size,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory('data/validation',
                                                   target_size=(hx, hy), 
                                                   batch_size=batch_size,
                                                    class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [30]:
model.save("cat-dog-vanilla-cnn-model.h5")

In [22]:
model.fit_generator(train_generator, steps_per_epoch=2000 // batch_size, epochs=50, validation_data=validation_generator, validation_steps=800 // batch_size)
model.save_weights('dog-cat-classifier.h5')

Epoch 1/50
125/125 [==============================] - 79s 628ms/step - loss: 0.7159 - acc: 0.4920 - val_loss: 0.6911 - val_acc: 0.5225
Epoch 2/50
125/125 [==============================] - 66s 525ms/step - loss: 0.6872 - acc: 0.5390 - val_loss: 0.6739 - val_acc: 0.5587
Epoch 3/50
125/125 [==============================] - 67s 538ms/step - loss: 0.6834 - acc: 0.5625 - val_loss: 0.6516 - val_acc: 0.6162
Epoch 4/50
125/125 [==============================] - 65s 520ms/step - loss: 0.6809 - acc: 0.5720 - val_loss: 0.6470 - val_acc: 0.5387
Epoch 5/50
125/125 [==============================] - 71s 567ms/step - loss: 0.6638 - acc: 0.6035 - val_loss: 0.6451 - val_acc: 0.6175
Epoch 6/50
125/125 [==============================] - 74s 588ms/step - loss: 0.6784 - acc: 0.5995 - val_loss: 0.6063 - val_acc: 0.6737
Epoch 7/50
125/125 [==============================] - 74s 596ms/step - loss: 0.6467 - acc: 0.6300 - val_loss: 0.6112 - val_acc: 0.6763
Epoch 8/50
125/125 [==============================] - 7

In [24]:
from keras import backend
from keras.models import load_model

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

def export_h5_to_pb(path_to_h5, export_path):

    # Set the learning phase to Test since the model is already trained.
    backend.set_learning_phase(0)

    # Load the Keras model
    keras_model = load_model(path_to_h5)

    # Build the Protocol Buffer SavedModel at 'export_path'
    builder = saved_model_builder.SavedModelBuilder(export_path)

    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(inputs={"images": keras_model.input},
                                      outputs={"scores": keras_model.output})

    with K.get_session() as sess:
        # Save the meta graph and the variables
        builder.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                         signature_def_map={"predict": signature})

    builder.save()

In [31]:
export_h5_to_pb("cat-dog-vanilla-cnn-model.h5", "cat-dog-vanilla-cnn.pb")

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'cat-dog-vanilla-cnn.pb/saved_model.pb'
